In [5]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from multiprocessing import Process, Pool
from threading import Thread
import time
%matplotlib inline

# Nacteni dat

In [6]:
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/items.json",'r') as f:
    items = json.loads(f.read())

with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/properties.json",'r') as f:
    properties = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/user.folds.json",'r') as f:
    user_folds = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/users.int2str.json",'r') as f:
    users_int2str = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/users.str2int.json",'r') as f:
    users_str2int = json.loads(f.read())
      
ratings = pd.read_csv('/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/ratings.csv')

with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/items.int2str.json",'r') as f:
    items_int2str = json.loads(f.read())
    
with open("/home/kuba/ownCloud/ModGen-fac-mat/sample-data-v2/movielens_1m/items.str2int.json",'r') as f:
    items_str2int = json.loads(f.read())

# Faktorizace

In [7]:
import multiprocessing
import ctypes
import numpy as np

In [8]:
import numpy as np
from numpy.linalg import inv
# %load_ext line_profiler

In [9]:
np.dot?

In [10]:
class MatrixFactorization:
    def __init__(self, ratings, no_factors):
        self.Ratings = ratings.pivot_table(columns=['userId'],index=['itemId'],values='rating')
        self.Ratings_matrix = np.nan_to_num(self.Ratings.values)

        self.Idx = np.isfinite(self.Ratings.values)

        self.ratings_dict = dict(ratings.apply(lambda r:( (str(int(r.userId)),str(int(r.itemId))),{
                                                                    'rating':r.rating,
                                                                     }), axis=1).values)
        self.no_factors = no_factors
        self.Users = None
        self.Items = None
        
    def init(self):
        if(self.weights_mode == "AllRank"):
            self.Weights = np.isfinite(self.Ratings.values).astype(np.float64, copy=False)
            self.Weights[self.Weights==0] = self.weight
            
        elif(self.weights_mode == "AllRank-pop"):
            pass
        elif(self.weights_mode == "MF-RMSE"):
            self.Weights = np.isfinite(self.Ratings.values).astype(np.float64, copy=False)

            
        self.Ratings_matrix += self.imputation_value
        if(self.random_init): 
            shared_array_base = multiprocessing.Array(ctypes.c_double, np.random.rand(self.Ratings.shape[1]* self.no_factors), lock=False)
            shared_array = np.frombuffer(shared_array_base,dtype=float)    
            self.Users = shared_array.reshape(self.Ratings.shape[1], self.no_factors)

            shared_array_base = multiprocessing.Array(ctypes.c_double, np.random.rand(self.Ratings.shape[0]* self.no_factors), lock= False)
            shared_array = np.frombuffer(shared_array_base,dtype=float)
            self.Items = shared_array.reshape(self.Ratings.shape[0], self.no_factors)
        
    '''
    RMSE
    '''  
    def get_error(self, R, U, V, W):
        return np.sum((W * (R - np.dot(V, U.T)))**2) 
    
    '''
    OPTIMAZE
    '''
    
    def compute_item_factor(self, batch):

        W, R, V, U, Idx = self.Weights, self.Ratings_matrix, self.Items, self.Users, self.Idx
        lambda_, alfa, opt_method,r_m = self.lambda_, self.alfa, self.opt_method, self.imputation_value
        
        if(opt_method == "solver"):
            for i in batch:
#                 V[i] = np.linalg.solve(np.dot(U.T, np.dot(np.diag(Wi), U)) + lambda_ * np.eye(self.no_factors),
#                                              np.dot(U.T, np.dot(np.diag(Wi), R[:, i])))
                lM = np.dot(np.dot((R[i,:] - r_m),np.diag(W[i,:])),U)
                rM = np.dot(np.dot(U.T,np.diag(W[i,:])),U) + lambda_ * np.dot(np.trace(np.diag(W[i,:])),np.eye(self.no_factors))
#                 V[i,:] = np.dot(lM, inv(rM))
                V[i,:] = np.linalg.solve(rM,lM)
        if(opt_method == "gradient-descent"):
            for i in batch:
                for c in range(50):
                    V[i] += -alfa * (np.dot((np.dot(V[i], U[Idx[i,:],:].T) - R[i,Idx[i,:]]), U[Idx[i,:],:]) + lambda_ * V[i])
        return
    
    def compute_user_factor(self, batch):
        
        W, R, V, U, Idx = self.Weights, self.Ratings_matrix, self.Items, self.Users, self.Idx
        lambda_, alfa, opt_method,r_m = self.lambda_, self.alfa, self.opt_method, self.imputation_value
        
        if(opt_method == "solver"):
            for u in batch:
#                 U[u] = np.linalg.solve(np.dot(V.T, np.dot(np.diag(Wu), V)) + lambda_ * np.eye(self.no_factors),
#                                            np.dot(V.T, np.dot(np.diag(Wu), R[u].T))).T
                lM = np.dot(np.dot(R[:,u].T - r_m, np.diag(W[:,u])), V)
                rM = np.dot(np.dot(V.T,np.diag(W[:,u])), V) + lambda_ * np.dot(np.trace(np.diag(W[:,u])), np.eye(self.no_factors))
#                 U[u,:] = np.dot(lM, inv(rM))
                U[u,:] = np.linalg.solve(rM,lM)
        if(opt_method == "gradient-descent"):
             for u in batch:
                for c in range(50):
                    U[u] += -alfa * (np.dot((np.dot(U[u], V[Idx[:,u],:].T) - R[Idx[:,u], u].T), V[Idx[:,u],:]) + lambda_* U[u] )
    
        return
    
    def optimize_rmse(self):
        
        self.init()
        weighted_errors = []

        step_item = int(np.ceil(len(self.Items)/float(self.no_process)))
        step_user = int(np.ceil(len(self.Users)/float(self.no_process)))
        
        item_range = [range(i,min(len(self.Items), i+step_item)) for i in range(0, len(self.Items), step_item)]
        user_range = [range(i,min(len(self.Users), i+step_user)) for i in range(0, len(self.Users), step_user)]
        print("Item range", item_range)
        print("User range", user_range)
        
        for ii in range(self.no_iterations):
            #ITEMS                
            process = []
            for batch in item_range:
                p = Process(target = self.compute_item_factor, args = (batch,))
                process.append(p)
                p.daemon = True
                p.start()
            
            [p.join() for p in process]
                
            #USERS
            process = []
            for batch in user_range:
                p = Process(target = self.compute_user_factor, args = (batch,))
                process.append(p)
                p.daemon = True
                p.start()
            
            [p.join() for p in process]

            if(not ii % 2):
                weighted_errors.append(self.get_error(self.Ratings_matrix, self.Users, self.Items, self.Weights))
                print('{}th iteration is completed'.format(ii))
                print(weighted_errors[-1])
                if(weighted_errors[-1] < 241530):
                    self.R_hat = np.dot(self.Users, self.Items.T)
                    return weighted_errors
            
                
        return weighted_errors
    
    def plot_rmse(self, weighted_errors):
        plt.plot(np.log(weighted_errors), label="weighted error: "+str(weighted_errors[-1]))
        plt.ylabel("RMSE log scale")
        plt.xlabel("no iterations")

        plt.legend()
        plt.show()
        
    def optimaze(self, no_iterations, loss_function="RMSE", opt_method="solver",  lambda_ = 0.001, alfa=0.00001, no_process = 1, random_init = True, weights_mode = "MF-RMSE", weight = 0, imputation_value = 0):
        self.loss_function = loss_function
        self.solve_opt_method = opt_method 
        self.lambda_ = lambda_
        self.alfa = alfa
        self.no_process = no_process
        self.no_iterations = no_iterations
        self.random_init = random_init
        self.weights_mode = weights_mode
        self.weight = weight
        self.imputation_value = imputation_value
        self.opt_method = opt_method
        
        weighted_errors = self.optimize_rmse()
        self.plot_rmse(weighted_errors)
       

In [11]:
# %lprun -f MFact.rmse_optimalization MFact.rmse_optimalization(0.001, 0.0001, 5)

In [12]:
import datetime
MFact = MatrixFactorization(ratings.copy(), 10)

In [13]:
start_time = datetime.datetime.now()
MFact.optimaze(400, opt_method="solver", lambda_ = 0.0001,  alfa = 0.0001, no_process = 8, weights_mode = "MF-RMSE", weight = 0, random_init = True)
datetime.datetime.now() - start_time

Item range [range(0, 464), range(464, 928), range(928, 1392), range(1392, 1856), range(1856, 2320), range(2320, 2784), range(2784, 3248), range(3248, 3706)]
User range [range(0, 755), range(755, 1510), range(1510, 2265), range(2265, 3020), range(3020, 3775), range(3775, 4530), range(4530, 5285), range(5285, 6040)]


Process Process-14:
Traceback (most recent call last):
  File "/usr/lib/python3.4/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.4/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-70d15564aca9>", line 75, in compute_user_factor
    rM = np.dot(np.dot(V.T,np.diag(W[:,u])), V) + lambda_ * np.dot(np.trace(np.diag(W[:,u])), np.eye(self.no_factors))
  File "/usr/local/lib/python3.4/dist-packages/numpy/lib/twodim_base.py", line 307, in diag
    res[:n-k].flat[i::n+1] = v
KeyboardInterrupt
Process Process-9:
Process Process-15:
KeyboardInterrupt


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/lib/python3.4/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.4/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-70d15564aca9>", line 75, in compute_user_factor
    rM = np.dot(np.dot(V.T,np.diag(W[:,u])), V) + lambda_ * np.dot(np.trace(np.diag(W[:,u])), np.eye(self.no_factors))
  File "/usr/local/lib/python3.4/dist-packages/numpy/lib/twodim_base.py", line 307, in diag
    res[:n-k].flat[i::n+1] = v
KeyboardInterrupt
Process Process-13:
Traceback (most recent call last):
  File "/usr/lib/python3.4/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/lib/python3.4/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-10-70d15564aca9>", line 74, in compute_user_factor
    lM = np.dot(np.dot(R[:,u].T - r_m, np.diag(W[:,u])), V)
  File "/usr/loc